In [65]:
import numpy as np
import pandas as pd
#from haversine import haversine, Unit
import geopy.distance
from tqdm import tqdm

In [89]:
small_df = pd.read_csv('../DAOU/test_small_out_v1.csv', encoding='euc-kr')
small_df.drop(small_df.columns[0], axis=1, inplace = True)
small_df['네이버 이런점이 좋아요 {항목:좋아요수}'].notnull
small_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    610 non-null    object 
 1   업소유형                    251 non-null    object 
 2   관광숙박업상세명                319 non-null    object 
 3   건물용도명                   209 non-null    object 
 4   지번주소                    610 non-null    object 
 5   도로명주소                   602 non-null    object 
 6   위도                      583 non-null    float64
 7   경도                      583 non-null    float64
 8   별점                      172 non-null    float64
 9   방문자 리뷰수                 231 non-null    object 
 10  블로그 리뷰수                 229 non-null    object 
 11  지하철역과의 거리               177 non-null    object 
 12  도보시간                    37 non-null     object 
 13  객실수                     307 non-null    float64
 14  시설면적                    358 non-null    fl

In [90]:
subway = pd.read_csv('../DABA/서울시 역사마스터 정보.csv', index_col=0, encoding='euc-kr')
subway.reindex()
subway

,역사명,호선,위도,경도
역사_ID,,,,
9996,미사,5호선,127.193877,37.560927
9995,강일,5호선,127.175930,37.557490
4929,김포공항,김포골드라인,126.801868,37.562360
4928,고촌,김포골드라인,126.770345,37.601243
4927,풍무,김포골드라인,126.732387,37.612488
...,...,...,...,...
154,종로5가,1호선,127.001849,37.570926
153,종로3가,1호선,126.991847,37.570406
152,종각,1호선,126.982923,37.570161


In [91]:
small_df.iloc[0]['사업장명']

print(np.nan)

nan


In [70]:
# .. 가까운 지하철 거리 및 도보 시간 구하기
# 가까운 지하철 구하기

small_dist = []

for i in tqdm(range(small_df.shape[0])):
#for i in range(1):
    dummy = []

    if np.isnan(small_df.iloc[i]['위도']) :

        dummy.append(np.nan)
        find_min_j = np.nan
        find_stn_nam = np.nan
        find_stn_lat = np.nan
        find_stn_lon = np.nan
        nearest_stn = np.nan
        time_to_near = np.nan

    else:

        for j in range(subway.shape[0]):

            hotel_latlon = (small_df.iloc[i]['위도'], small_df.iloc[i]['경도'])
            subway_latlon = (subway.iloc[j]['경도'], subway.iloc[j]['위도'])    # 원본데이터에서 위경도 컬럼명 바뀌어 있음

            dist = geopy.distance.geodesic(hotel_latlon, subway_latlon).m
            dummy.append(dist)

        find_min_j = np.argmin(dummy)
        dist_to_stn = np.nanmin(dummy)
        time_to_stn = np.nanmin(dummy)/66.7

        #print(find_min_j)
        find_stn_nam = subway.iloc[find_min_j]['역사명']
        find_stn_lat = subway.iloc[find_min_j]['위도']
        find_stn_lon = subway.iloc[find_min_j]['경도']

        
            
        #print(f"호텔:{small_df.iloc[i]['사업장명']} {small_df.iloc[i]['지번주소']} {min(dummy)}m")
    small_dist.append([small_df.iloc[i]['사업장명'],
                        small_df.iloc[i]['지번주소'],
                        small_df.iloc[i]['위도'],
                        small_df.iloc[i]['경도'],
                        find_stn_nam,
                        find_stn_lat,
                        find_stn_lon,
                        dist_to_stn,
                        time_to_stn]) # 시속 4km --> 분속 66.7m



100%|██████████| 610/610 [03:30<00:00,  2.90it/s]


In [92]:
find_small_dist = pd.DataFrame(small_dist, columns=['사업장명','지번주소','호텔 위도', '호텔 경도','역이름', '역 위도','역 경도', '거리','도보시간(분)'])

#for i in range(10):
#    print(f"호텔:{small_df.iloc[i]['사업장명']}, {df.iloc[i]['거리']}")
find_small_dist.info()
find_small_dist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     610 non-null    object 
 1   지번주소     610 non-null    object 
 2   호텔 위도    583 non-null    float64
 3   호텔 경도    583 non-null    float64
 4   역이름      583 non-null    object 
 5   역 위도     583 non-null    float64
 6   역 경도     583 non-null    float64
 7   거리       610 non-null    float64
 8   도보시간(분)  610 non-null    float64
dtypes: float64(6), object(3)
memory usage: 43.0+ KB


,사업장명,지번주소,호텔 위도,호텔 경도,역이름,역 위도,역 경도,거리,도보시간(분)
0,호텔 데님(Hotel Denim),서울특별시 강남구 개포동 1229-14,37.476190,127.046616,양재시민의숲(매헌),127.038420,37.470023,997.016642,14.947776
1,티마크그랜드호텔명동,서울특별시 중구 회현동1가 194-15,37.558454,126.978646,회현(남대문시장),126.978246,37.558514,35.993514,0.539633
2,위너스,서울특별시 금천구 독산동 335-21,37.471959,126.889817,독산,126.889249,37.466613,595.471106,8.927603
3,데이즈 호텔 명동,서울특별시 중구 충무로1가 24-30,37.560953,126.983631,명동,126.986325,37.560989,238.049587,3.568959
4,강남패밀리호텔,서울특별시 강남구 논현동 205-8,37.506474,127.030156,언주,127.033868,37.507287,340.365147,5.102926
...,...,...,...,...,...,...,...,...,...
605,하모니관광호텔,서울특별시 강동구 길동 387-1번지,37.538231,127.138108,길동,127.140004,37.537801,174.229810,2.612141
606,호텔 감(HOTEL GAM),서울특별시 강동구 천호동 452-30번지,37.538400,127.128669,강동,127.132481,37.535804,443.301249,6.646196
607,토요코인 서울 동대문 Ⅱ,서울특별시 중구 광희동2가 14-1,37.564521,127.007868,동대문역사문화공원,127.007885,37.565133,67.907433,1.018102
608,발리(BALI),서울특별시 강동구 성내동 417-5,37.529051,127.134907,둔촌동,127.136248,37.527788,183.596511,2.752571


In [93]:
for i,j,k in zip(find_small_dist['역이름'], find_small_dist['도보시간(분)'], find_small_dist['거리']):
    print(i,j,k)

양재시민의숲(매헌) 14.947775737718288 997.0166417058099
회현(남대문시장) 0.5396328900957053 35.99351376938354
독산 8.92760279587299 595.4711064847285
명동 3.568959324794976 238.0495869638249
언주 5.102925737318967 340.3651466791751
용산 4.16280709627942 277.65923332183735
회현(남대문시장) 5.6629344100817764 377.7177251524545
을지로3가 4.90859776216487 327.40347073639686
양평 6.260499970723792 417.575348047277
학동 6.258818241037127 417.46317667717636
용산 3.660254713910429 244.13898941782563
영등포시장 5.848335929900764 390.084006524381
잠실새내 6.113743097221603 407.7866645846809
충무로 1.675462595284765 111.75335510549384
명동 4.422199066379994 294.9606777275456
신림 2.0705673413504293 138.10684166807363
신림 2.2334136709148904 148.9686918500232
신림 1.4050764500698782 93.71859921966089
마곡나루 2.2421729321502246 149.55293457442
홍대입구 3.2731103904184886 218.3164630409132
구로디지털단지 4.110844951504392 274.19335826534297
서대문 1.039747948159865 69.351188142263
가산디지털단지 11.357999446054233 757.5785630518174
명동 4.181523639810771 278.90762677537845
선릉 3.90079

In [94]:
small_df['지하철역과의 거리'] = find_small_dist['거리']
small_df['도보시간'] = find_small_dist['도보시간(분)']
small_df['가까운 지하철역'] = find_small_dist['역이름']
#find_small_dist['역이름']
small_df['가까운 지하철역']

0      양재시민의숲(매헌)
1       회현(남대문시장)
2              독산
3              명동
4              언주
          ...    
605            길동
606            강동
607     동대문역사문화공원
608           둔촌동
609           답십리
Name: 가까운 지하철역, Length: 610, dtype: object

In [95]:
small_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    610 non-null    object 
 1   업소유형                    251 non-null    object 
 2   관광숙박업상세명                319 non-null    object 
 3   건물용도명                   209 non-null    object 
 4   지번주소                    610 non-null    object 
 5   도로명주소                   602 non-null    object 
 6   위도                      583 non-null    float64
 7   경도                      583 non-null    float64
 8   별점                      172 non-null    float64
 9   방문자 리뷰수                 231 non-null    object 
 10  블로그 리뷰수                 229 non-null    object 
 11  지하철역과의 거리               610 non-null    float64
 12  도보시간                    610 non-null    float64
 13  객실수                     307 non-null    float64
 14  시설면적                    358 non-null    fl

In [96]:
small_df['가까운 지하철역']

0      양재시민의숲(매헌)
1       회현(남대문시장)
2              독산
3              명동
4              언주
          ...    
605            길동
606            강동
607     동대문역사문화공원
608           둔촌동
609           답십리
Name: 가까운 지하철역, Length: 610, dtype: object

In [97]:
colum_names2 = ['사업장명','업소유형','관광숙박업상세명','건물용도명','지번주소','도로명주소','가까운 지하철역', '지하철역과의 거리', '도보시간',
                '위도','경도', '별점','방문자 리뷰수', '블로그 리뷰수', '객실수','시설면적','시설규모','구비시설',
                '네이버 이런점이 좋아요 총합', '네이버 이런점이 좋아요 {항목:좋아요수}', '데이터갱신일자',
                '인허가일자','인허가취소일자','영업상태코드','폐업일자','휴업시작일자','휴업종료일자','재개업일자']

small_df = small_df[colum_names2]
small_df

,사업장명,업소유형,관광숙박업상세명,건물용도명,지번주소,도로명주소,가까운 지하철역,지하철역과의 거리,도보시간,위도,...,네이버 이런점이 좋아요 총합,네이버 이런점이 좋아요 {항목:좋아요수},데이터갱신일자,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자
0,호텔 데님(Hotel Denim),2성급,NaN,NaN,서울특별시 강남구 개포동 1229-14,서울특별시 강남구 논현로 66 (개포동),양재시민의숲(매헌),997.016642,14.947776,37.476190,...,23.0,"{'깨끗해요': 9},{'친절해요': 7},{'주차하기 편해요': 7},{'조용히 ...",2022-12-02 22:09:00.0,2015-02-03,NaN,1,NaN,NaN,NaN,NaN
1,티마크그랜드호텔명동,뷔페,NaN,NaN,서울특별시 중구 회현동1가 194-15,"서울특별시 중구 퇴계로 52 (회현동1가, 인송빌딩)",회현(남대문시장),35.993514,0.539633,37.558454,...,NaN,NaN,2022-12-03 00:03:00.0,2016-05-17,NaN,3,2023-02-28,NaN,NaN,NaN
2,위너스,NaN,NaN,NaN,서울특별시 금천구 독산동 335-21,서울특별시 금천구 가산로3길 103 (독산동),독산,595.471106,8.927603,37.471959,...,NaN,NaN,2022-12-02 23:02:00.0,2021-04-06,NaN,1,NaN,NaN,NaN,NaN
3,데이즈 호텔 명동,3성급,NaN,NaN,서울특별시 중구 충무로1가 24-30,서울특별시 중구 퇴계로 107 (충무로1가),명동,238.049587,3.568959,37.560953,...,NaN,NaN,2022-12-02 23:05:00.0,2014-07-22,NaN,1,NaN,NaN,NaN,NaN
4,강남패밀리호텔,3성급,NaN,NaN,서울특별시 강남구 논현동 205-8,"서울특별시 강남구 봉은사로 143 (논현동, 운현오피스텔)",언주,340.365147,5.102926,37.506474,...,16.0,"{'깨끗해요': 6},{'조용히 쉬기 좋아요': 5},{'대중교통이 편해요': 3}...",2022-12-03 23:02:00.0,2016-03-29,NaN,2,NaN,2023-01-01,2023-12-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,하모니관광호텔,NaN,관광호텔업,NaN,서울특별시 강동구 길동 387-1번지,서울특별시 강동구 천중로40길 63 (길동),길동,174.229810,2.612141,37.538231,...,NaN,NaN,2020-02-16 02:40:00.0,20140724,NaN,1,NaN,NaN,NaN,NaN
606,호텔 감(HOTEL GAM),NaN,관광호텔업,NaN,서울특별시 강동구 천호동 452-30번지,서울특별시 강동구 천호대로159길 22-36 (천호동),강동,443.301249,6.646196,37.538400,...,NaN,NaN,2020-02-16 02:40:00.0,20171226,NaN,1,NaN,NaN,NaN,NaN
607,토요코인 서울 동대문 Ⅱ,NaN,NaN,NaN,서울특별시 중구 광희동2가 14-1,서울특별시 중구 퇴계로 325 (광희동2가),동대문역사문화공원,67.907433,1.018102,37.564521,...,NaN,NaN,2021-12-05 00:08:00.0,20181120,NaN,1,NaN,NaN,NaN,NaN
608,발리(BALI),호텔,NaN,NaN,서울특별시 강동구 성내동 417-5,서울특별시 강동구 풍성로 210 (성내동),둔촌동,183.596511,2.752571,37.529051,...,20.0,"{'친절해요': 9},{'깨끗해요': 8},{'조용히 쉬기 좋아요': 5},{'즐길...",2022-11-30 23:04:00.0,20030128,NaN,1,NaN,NaN,NaN,NaN


In [98]:
output_path = "../DAOU/test_small_out_v2.csv"
small_df.to_csv(output_path, encoding='euc-kr')